# 0. Install dependencies

In [ ]:
# Install PyTorch
!pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install sentencepiece

In [1]:
# Install transformers
!pip install transformers

In [2]:
#Install Beautiful soup
!pip install Beautifulsoup4
!pip install requests

# 1. Import and Load Model

In [9]:
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

#Import Scraper
from bs4 import BeautifulSoup as bs
import requests
import re

#Import CSV viewer
import pandas as pd


This below cell is for google colab dependencies

In [10]:
from google.colab import drive
from google.colab import files 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
#Change this to the CSV path
csv = pd.read_csv('gdrive/MyDrive/Colab Notebooks/STOVE/stackoverflow_post_summary_groundtruth.csv')

In [86]:
# Load tokenizer 
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-reddit_tifu") #Need to try different pretrained data sets

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

In [87]:
# Load model 
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-reddit_tifu")

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

# 2. Scrape for question and answers

This cell scrapes from the CSV file given in the upload cell above for relevant information.

In [88]:
def ScrapeCSV(csv_in):

  code = []
  mod_answer_text = []
  unmod_answer_text = []
  data = []

  #This regex pattern pulls everything between the code tags
  pat_c = r'(?=\<code>).+?(?<=\</code>)'

  #This regex pattern extracts the tags alltogether.
  pat_t = r'(?=\<).+?(?<=\>)'


  #for i in range len(csv_in):
  for answer in csv_in['answer_body']:
    answer = answer[1:-1]
    answer = answer.replace('\'','')
    answer = answer.replace(',','')

    #This extracts the code from the answer, joins it, and appends it to code
    code_text = re.findall(pat_c, answer)
    code.append(' '.join(code_text))

    #This modifies the answer string to be without the tags but code remaining
    unmod_ans = re.sub(pat_t,'', answer)
    unmod_answer_text.append(unmod_ans)

    #This removes the code from the unmodified answer leaving just the text
    mod_ans = re.sub(pat_c, '', answer)
    mod_answer_text.append(mod_ans)

  data.append(code)
  data.append(unmod_answer_text)
  data.append(mod_answer_text)
  #Data is a tuple containing the code and both answer text arrays
  return data

  



In [89]:
#Test Cell
ScrapeCSV(csv)

[['<code>init_attributes</code>',
  '',
  '',
  '<code>type</code> <code>type</code> <code>type</code> <code>type</code> <code>__init__</code> <code>__new__</code> <code>class</code> <code>class</code> <code>__metaclass__</code> <code>__metaclass__</code> <code>type.__subclasses__()</code> <code>type</code> <code>__add__</code> <code>__iter__</code> <code>__getattr__</code>',
  '',
  '',
  '',
  '',
  '',
  '<code>print_args</code>',
  '<code>re.VERBOSE</code>',
  '',
  '<code>attrgetter()</code> <code>itemgetter()</code> <code>operator</code>',
  '',
  '',
  '',
  '<code>dlsym()</code> <code>dlopen()</code>',
  '<code>INNER JOIN</code> <code>CROSS APPLY</code> <code>CROSS APPLY</code> <code>JOIN</code> <code>3</code> <code>t2</code> <code>t1</code> <code>INNER JOIN</code> <code>CTE</code> <code>master</code> <code>20000000</code> <code>PRIMARY KEY</code> <code>id</code> <code>30</code>',
  '',
  '',
  '',
  '',
  '',
  '',
  '<code>dict.fromkeys()</code>',
  '<code>@classmethod</code>

This Cell was a test cell for web-based scraping with beautifulsoup. DO NOT USE with CSV dataset.


In [ ]:
#Scrape post
post_link = 'https://stackoverflow.com/questions/53621696/how-to-start-from-second-index-for-for-loop/53621801'
link = post_link
results = requests.get(link)
doc = bs(results.text, 'html.parser')

#This parses specifically for question text
text_tags = doc.find_all('div', itemprop='text')#, itemprop='text')

if (len(text_tags[0].text.split('\r')) == 1): #If there is not community lock message the question is the first element in the list
  question_text = text_tags[0].text.split('\r')[0] #the actual text is the 3rd element in the html tag

else:
  question_text = text_tags[0].text.split('\r')[2] #If there is a community lock message it will be the third element for some reason

##Strips random HTML and \n characters out of the text_tag
question_text = question_text.replace('\n',' ')
question_text = question_text.strip()

#question_text
answer_texts = []
#This parses for the various answer text_tags and strips out random characters
for i in range(1, len(text_tags)):
    answer_texts.append(text_tags[i].text)
    answer_texts[i-1] = answer_texts[i-1].replace('\n', ' ')
    answer_texts[i-1] = answer_texts[i-1].strip()

answer_texts
    

In [ ]:
text = answer_texts[0]

#3. Perform Abstractive Summarization

In [97]:
#This is for test purposes and is only extracting the first unmod answer text
text = ScrapeCSV(csv)[1][3]
text

'A metaclass is the class of a class. Like a class defines how an instance of the class behaves a metaclass defines how a class behaves. A class is an instance of a metaclass. While in Python you can use arbitrary callables for metaclasses (like Jerub shows) the more useful approach is actually to make it an actual class itself. type is the usual metaclass in Python. "In case youre wondering yes type is itself a class and it is its own type." "You wont be able to recreate something like type purely in Python but Python cheats a little." To create your own metaclass in Python you really just want to subclass type. A metaclass is most commonly used as a class-factory. "Like you create an instance of the class by calling the class Python creates a new class (when it executes the class statement) by calling the metaclass." "Combined with the normal __init__ and __new__ methods metaclasses therefore allow you to do extra things when creating a class like registering the new class with some 

In [99]:
# Create tokens - number representation of our text
tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")

In [100]:
# Summarize - This can take a few minutes
summary = model.generate(**tokens)

In [101]:
# Decode summary
tokenizer.decode(summary[0])

'metaclasses define the type of a class not just a factory for it but they allow you to do extra things when creating a class like registering the new class with some registry or even replace the class with something else entirely.'